<a href="https://colab.research.google.com/github/Melvinchen0404/Chess_puzzles/blob/main/puzzle_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**STEP 1:** Upload the compressed '`lichess_db_puzzle.csv.zst`' folder, decompress the folder, and inspect the first few lines of data

In [1]:
!pip install zstandard

import zstandard as zstd

# Decompress the .zst file
compressed_file = '/content/lichess_db_puzzle.csv.zst'
decompressed_file = '/content/lichess_db_puzzle.csv'

with open(compressed_file, 'rb') as f:
    with open(decompressed_file, 'wb') as out_f:
        dctx = zstd.ZstdDecompressor()
        dctx.copy_stream(f, out_f)

print(f"Decompressed file saved as {decompressed_file}")

# Inspect the decompressed file
with open(decompressed_file, 'rb') as f:
    # Read the first few bytes to guess the format
    data = f.read(1024)
    print(data)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 36.6 MB/s eta 0:00:00
Decompressed file saved as /content/lichess_db_puzzle.csv
b'PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags\n00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1902,76,95,7226,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,\n0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - - 2 27,d3d6 f8d8 d6d8 f6d8,1512,74,96,29629,advantage endgame short,https://lichess.org/F8M8OS71#53,\n0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1300,75,90,666,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,\n0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP1P/RN1QR1K1 b - - 1 16,b6c5 e2g4 h3g4 d1g4,1078,74,87,573,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Variation\n000VW,r4r2/1p3pkp/p5p1/3R1N1Q/3P4/8/P1q2P2/3R2K1 b -

**STEP 2:** Determine the total number of puzzles and the range of values for certain fields ('`Rating`', '`Popularity`', '`Themes`')

In [2]:
import pandas as pd

# Load the dataset into a DataFrame
data_file = "/content/lichess_db_puzzle.csv"
columns = [
    "PuzzleId", "FEN", "Moves", "Rating", "RatingDeviation",
    "Popularity", "NbPlays", "Themes", "GameUrl", "OpeningTags"
]

# Read the file into a pandas DataFrame
df = pd.read_csv(data_file, names=columns)

# Convert numeric fields to numbers, coercing errors to NaN
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Popularity"] = pd.to_numeric(df["Popularity"], errors="coerce")

# Transform negative Popularity values to positive
df["Popularity"] = df["Popularity"].abs()

# Total number of puzzles
total_puzzles = len(df)

# Range of rating values
rating_min = df["Rating"].min()
rating_max = df["Rating"].max()

# Range of popularity values
popularity_min = df["Popularity"].min()
popularity_max = df["Popularity"].max()

# Extract unique themes
all_themes = set()
df["Themes"].dropna().apply(lambda x: all_themes.update(x.split(" ")))

# Display results
print(f"Total number of puzzles: {total_puzzles}")
print(f"Rating range: {rating_min} - {rating_max}")
print(f"Popularity range: {popularity_min} - {popularity_max}")
print(f"Different themes available ({len(all_themes)}): {sorted(all_themes)}")

<ipython-input-2-9ce8e7f477f0>:11: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file, names=columns)


Total number of puzzles: 375766
Rating range: 399.0 - 3153.0
Popularity range: 0.0 - 100.0
Different themes available (61): ['Themes', 'advancedPawn', 'advantage', 'anastasiaMate', 'arabianMate', 'attackingF2F7', 'attraction', 'backRankMate', 'bishopEndgame', 'bodenMate', 'capturingDefender', 'castling', 'clearance', 'crushing', 'defensiveMove', 'deflection', 'discoveredAttack', 'doubleBishopMate', 'doubleCheck', 'dovetailMate', 'enPassant', 'endgame', 'equality', 'exposedKing', 'fork', 'hangingPiece', 'hookMate', 'interference', 'intermezzo', 'kingsideAttack', 'knightEndgame', 'long', 'master', 'masterVsMaster', 'mate', 'mateIn1', 'mateIn2', 'mateIn3', 'mateIn4', 'mateIn5', 'middlegame', 'oneMove', 'opening', 'pawnEndgame', 'pin', 'promotion', 'queenEndgame', 'queenRookEndgame', 'queensideAttack', 'quietMove', 'rookEndgame', 'sacrifice', 'short', 'skewer', 'smotheredMate', 'superGM', 'trappedPiece', 'underPromotion', 'veryLong', 'xRayAttack', 'zugzwang']


**STEP 3:** Create a widget and select chess puzzles from the puzzle database to solve

In [3]:
!pip install ipython ipywidgets python-chess pandas

import pandas as pd
import chess
import chess.svg
from IPython.display import display, SVG
import ipywidgets as widgets

# Load the cleaned dataset
data_file = "/content/lichess_db_puzzle.csv"  # Ensure this file contains the cleaned data
columns = [
    "PuzzleId", "FEN", "Moves", "Rating", "RatingDeviation",
    "Popularity", "NbPlays", "Themes", "GameUrl", "OpeningTags"
]
df = pd.read_csv(data_file, names=columns)

# Convert numeric fields
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Popularity"] = pd.to_numeric(df["Popularity"], errors="coerce")
df["Popularity"] = df["Popularity"].abs()  # Ensure Popularity is positive

# List of allowed themes
allowed_themes = [
    "advancedPawn", "discoveredAttack", "enPassant", "doubleBishopMate",
    "fork", "mateIn1", "mateIn2", "mateIn3", "mateIn4", "mateIn5",
    "skewer", "sacrifice", "underPromotion", "zugzwang"
]

# Function to display a chess board from FEN
def display_chess_board(fen, size=0.5):
    board = chess.Board(fen)
    return chess.svg.board(board=board, size=500*size)

# Function to filter puzzles based on user input
def filter_puzzles(theme, min_rating, max_rating, min_popularity, max_popularity):
    filtered = df[
        (df["Rating"] >= min_rating) &
        (df["Rating"] <= max_rating) &
        (df["Popularity"] >= min_popularity) &
        (df["Popularity"] <= max_popularity) &
        (df["Themes"].str.contains(theme))
    ]
    return filtered

# Interactive widgets
theme_widget = widgets.Dropdown(
    options=["Any"] + sorted(set(" ".join(df["Themes"].dropna()).split(" ")).intersection(allowed_themes)),
    value="Any",
    description="Theme:",
    layout=widgets.Layout(width="300px")
)

rating_range = widgets.FloatRangeSlider(
    value=[399.0, 3284.0],
    min=399.0,
    max=3284.0,
    step=1.0,
    description="Rating:",
    continuous_update=False,
    layout=widgets.Layout(width="400px")
)

popularity_range = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=100,
    step=1.0,
    description="Popularity:",
    continuous_update=False,
    layout=widgets.Layout(width="400px")
)

reveal_button = widgets.Button(
    description="Reveal Answer",
    button_style="success",  # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width="150px")
)

output = widgets.Output()

# Global variable to store the current puzzle
current_puzzle = {"FEN": None, "Moves": None}

# Function to handle user input and update the board display
def update_display(change=None):
    global current_puzzle
    with output:
        output.clear_output()
        theme = theme_widget.value
        min_rating, max_rating = rating_range.value
        min_popularity, max_popularity = popularity_range.value

        # Filter puzzles
        if theme == "Any":
            theme = ""  # No filter for themes
        puzzles = filter_puzzles(theme, min_rating, max_rating, min_popularity, max_popularity)

        if puzzles.empty:
            print("No puzzles found for the selected criteria.")
            current_puzzle = {"FEN": None, "Moves": None}
        else:
            # Pick the first puzzle in the filtered result
            puzzle = puzzles.iloc[0]
            print(f"Puzzle ID: {puzzle['PuzzleId']}")
            print(f"Rating: {puzzle['Rating']}, Popularity: {puzzle['Popularity']}")
            print(f"Themes: {puzzle['Themes']}")
            # Render the chessboard
            svg = display_chess_board(puzzle["FEN"], size=0.5)
            display(SVG(svg))
            # Store the current puzzle
            current_puzzle = {"FEN": puzzle["FEN"], "Moves": puzzle["Moves"]}

# Function to reveal the answer
def reveal_answer(change=None):
    with output:
        if current_puzzle["Moves"]:
            print(f"Solution Moves: {current_puzzle['Moves']}")
        else:
            print("No puzzle selected to reveal the answer.")

# Bind update function to widget changes
theme_widget.observe(update_display, names="value")
rating_range.observe(update_display, names="value")
popularity_range.observe(update_display, names="value")
reveal_button.on_click(reveal_answer)

# Display widgets and output area
display(widgets.VBox([theme_widget, rating_range, popularity_range, reveal_button, output]))

# Initialize the display
update_display()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=647ab712ecce833c9ba109000d52e1b8763079dc12457260c9311d2bc634d904
  Stored in directory: /root/.cache/pip/wheels/f0/3f/76/8783033e8524d407e1bebaf72fdd3f3eba27e0c030e92bbd87
Successfully built chess


<ipython-input-3-28fa6e414061>:15: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_file, names=columns)
